In [1]:
import sys
sys.path.append("..")

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

from estimator.xgbclassifier import XGBClassifier

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
x = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [111]:
python_clf = XGBClassifier.make(
    n_estimators=100, 
    reg_lambda=1.0, 
    gamma=0.5,
    max_depth=3,
    num_class=3,
    n_jobs=2,
    objective='multi:softprob',
)

python_model = python_clf.fit(x_train, y_train)

python_result = python_model.predict(x_test)

In [4]:
spark = SparkSession.builder.appName('my_test').config('spark.jars', '../xgboost4j_2.12-1.6.1.jar,../xgboost4j-spark_2.12-1.6.1.jar,../scala-util/out/artifacts/scala_util_jar/scala-util.jar').getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jingmin1987/hadoop/spark-3.3.0/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jingmin1987/hadoop/hadoop-3.3.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


22/07/23 15:22:44 WARN Utils: Your hostname, JazyJojo resolves to a loopback address: 127.0.1.1; using 192.168.1.155 instead (on interface wifi0)
22/07/23 15:22:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/23 15:22:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [78]:
scala_clf = XGBClassifier.make(
    backend='scala',
    spark=spark,
    eta=0.1,
    reg_lambda=1.0, 
    gamma=0.5,
    max_depth=3,
    objective='multi:softprob',
    num_class=3,
    num_round=100,
    num_workers=2
).set_features_col('features') \
.set_label_col('label')

In [79]:
xy_train = x_train.assign(label=y_train)
xy_test = x_test.assign(label=y_test)

In [80]:
xy_train_spark = spark.createDataFrame(xy_train)

In [81]:
xy_train_spark.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- label: long (nullable = true)



In [82]:
xy_train_spark.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'label']

In [83]:
vector = scala_clf.transform(xy_train_spark, xy_train_spark.columns[:-1], 'label')

In [84]:
_ = scala_clf.fit(vector)

22/07/23 16:18:04 WARN XGBoostSpark: train_test_ratio is deprecated since XGBoost 0.82, we recommend to explicitly pass a training and multiple evaluation datasets by passing 'eval_sets' and 'eval_set_names'
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=192.168.1.155, DMLC_TRACKER_PORT=57625, DMLC_NUM_WORKER=2}


[16:18:04] task 0 got new rank 0
[16:18:04] task 1 got new rank 1


In [85]:
xy_test_spark = spark.createDataFrame(xy_test)
vector_test = scala_clf.transform(xy_test_spark, xy_test_spark.columns[:-1], 'label')

In [86]:
result = scala_clf.predict(vector_test)

In [87]:
from pyspark.sql import DataFrame

In [88]:
result_df = DataFrame(result, spark)

In [112]:
scala_result = result_df.select('prediction').toPandas()['prediction'].astype(int).to_numpy()

In [113]:
python_result == scala_result

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])